## LCEL 인터페이스


사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


LCEL 문법을 사용하여 chain 을 생성합니다.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

## stream: 실시간 출력


이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 

In [4]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

멀티모달은 여러 가지 다른 형태의 커뮤니케이션 매체를 조합하여 정보를 전달하는 방식을 의미합니다. 이는 텍스트, 음성, 그래픽, 영상 등 다양한 형태의 미디어를 활용하여 효과적인 커뮤니케이션을 구축할 수 있게 해줍니다. 이러한 다양한 매체를 활용함으로써 사용자들에게 더욱 다양하고 풍부한 경험을 제공할 수 있습니다.

## invoke: 호출


`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [5]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT 는 OpenAI에서 개발된 대화형 인공지능 모델로, 사용자와 자연스럽게 대화하며 질문에 답변하거나 다양한 주제에 대한 정보를 제공합니다. ChatGPT는 일상적인 대화뿐만 아니라 전문적인 지식과 감정을 다루기도 하며, 다양한 분야에서 활용될 수 있습니다. 그 결과로, ChatGPT는 현대 인공지능 기술을 획기적으로 발전시킨 대화형 모델 중 하나로 평가받고 있습니다.'

## batch: 배치(단위 실행)


함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다.

In [6]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 인공지능 챗봇으로, 자연어 처리 기술을 사용하여 대화를 합니다. 사용자의 질문이나 요청에 대해 정확한 정보와 답변을 제공하며, 학습을 통해 계속 발전하고 있습니다. 다양한 주제에 대해 대화를 나누고, 사용자와 자연스럽게 소통하는 데에 중점을 둔 서비스입니다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 삶과 관심사를 시각적으로 공유할 수 있습니다. 해시태그를 통해 다양한 주제나 관심사에 해당하는 콘텐츠를 찾을 수 있고, 팔로워나 좋아요 등을 통해 소통과 상호작용이 이뤄집니다. 인플루언서나 브랜드들도 활발히 활동하며, 팬들과의 소통을 통해 광고나 마케팅에 활용되기도 합니다.']

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

`config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [7]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 인공지능 챗봇으로 자연어 처리 기술을 활용하여 사용자와 대화합니다. 사용자의 질문에 답변하거나 대화를 이어가는데 사용되며 다양한 주제에 대해 대화할 수 있습니다. ChatGPT는 실시간 대화, 고객 서비스 및 정보 제공 등 다양한 분야에서 활용될 수 있습니다.',
 'Instagram은 사진과 동영상을 공유하며 다른 사람들과 소통하는 소셜 미디어 플랫폼이다. 전 세계적으로 많은 사람들이 사용하며 인기 있는 인플루언서나 브랜드들이 활발하게 활동하는 곳이기도 하다. 다양한 필터와 기능을 이용해 사용자들은 자신만의 창의적인 콘텐츠를 만들어 공유할 수 있다.',
 '멀티모달은 여러 가지 다른 형태의 커뮤니케이션을 결합하여 사용자 경험을 향상시키는 기술이다. 예를 들어 음성, 텍스트, 이미지, 동영상 등을 함께 사용하여 사용자가 보다 자연스럽게 상호작용할 수 있도록 돕는다. 멀티모달 기술은 인공지능, 컴퓨터 비전, 음성인식 등 다양한 기술을 활용하여 발전되고 있으며, 현재는 스마트폰, 스마트 홈 장치, 자율주행 자동차 등 다양한 분야에 적용되고 있다.',
 '프로그래밍은 컴퓨터에게 명령을 내리는 작업을 의미합니다. 이를 통해 사용자의 요구사항에 맞는 소프트웨어를 개발할 수 있습니다. 프로그래밍 언어를 사용하여 코드를 작성하고 실행함으로써 원하는 결과를 얻을 수 있습니다.',
 '머신러닝은 컴퓨터가 데이터를 분석하여 패턴을 학습하고 예측하는 인공지능 기술이다. \n\n머신러닝은 알고리즘을 사용하여 데이터를 처리하고 모델을 만들어 학습시키는 과정을 거친다.\n\n머신러닝은 주로 패턴 인식, 예측 및 의사 결정을 위해 사용되며, 데이터의 양과 질이 중요한 역할을 한다.']

## async stream: 비동기 스트림


함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print` 함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.


In [8]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 전 세계에서 가장 인기 있는 동영상 공유 플랫폼으로, 사용자들은 다양한 주제의 동영상을 시청하고 업로드할 수 있습니다.
유튜브를 통해 사용자들은 뮤직비디오, 블로그, 만화, 교육 콘텐츠 등 다양한 형태의 동영상을 즐길 수 있으며, 댓글을 통해 다른 사용자들과 소통할 수도 있습니다.
유튜브는 광고 수익, 후원자 및 프리미엄 구독자 등 다양한 수익 모델을 통해 크리에이터들에게 수익 창출의 기회를 제공합니다.

## async invoke: 비동기 호출


`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.


In [13]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

In [14]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

'NVDA는 엔비디아 코퍼레이션(NVIDIA Corporation)의 주식 코드이며, 미국의 그래픽 처리 장치를 생산하는 회사이다. 전 세계적으로 그래픽 카드 시장에서 선두를 달리고 있으며, 인공지능, 자율주행차 및 클라우드 컴퓨팅 분야에서도 선도적인 기술을 제공하고 있다. 특히 최근에는 딥러닝 분야에서의 성과로 주목받고 있으며, 고성능 그래픽 카드인 GeForce 및 Quadro 시리즈를 생산하고 있다.'

## async batch: 비동기 배치


함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 `chain` 객체의 `abatch` 메서드를 사용하여 `topic` 에 대한 작업을 비동기적으로 처리하고 있습니다.

`await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.


In [15]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [16]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process

['YouTube는 전 세계에서 가장 인기 있는 온라인 동영상 공유 플랫폼으로, 사용자들이 무료로 다양한 동영상을 시청하고 업로드할 수 있는 서비스를 제공합니다. 유튜버들은 자신의 채널을 통해 다양한 콘텐츠를 제작하고 수익을 창출할 수 있으며, 시청자들은 원하는 영상을 구독하고 댓글을 달아 소통할 수 있습니다. YouTube는 광고 수익을 통해 수익을 창출하며, 현재는 다양한 국가와 언어로 서비스되고 있습니다.',
 'Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 애플리케이션입니다. 사용자들은 팔로워들과 소통하며 일상 속의 다양한 순간을 공유할 수 있습니다. 필터 기능을 통해 사진을 가공하고 다양한 해시태그를 활용하여 사진을 탐색할 수 있습니다.',
 'Facebook은 사람들이 서로 소통하고 정보를 공유하는 소셜 네트워크 서비스이다. 사용자들은 글쓰기, 사진 및 동영상 공유, 댓글 등 다양한 기능을 통해 다른 사람들과 소통할 수 있다. 또한 광고 및 마케팅을 위한 플랫폼으로도 활용되고 있다.']

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때, 각 요소를 병렬로 실행합니다.


`langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

`ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `country`에 대한 **수도** 와 **면적** 을 구하는 두 개의 체인(`chain1`, `chain2`)을 만듭니다.

이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결됩니다. 마지막으로, `RunnableParallel` 클래스를 사용하여 이 두 체인을 `capital`와 `area`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성합니다.


In [17]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

`chain1.invoke()` 함수는 `chain1` 객체의 `invoke` 메서드를 호출합니다.

이때, `country`이라는 키에 `대한민국`라는 값을 가진 딕셔너리를 인자로 전달합니다.


In [18]:
# chain1 를 실행합니다.
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울이야.'

이번에는 `chain2.invoke()` 를 호출합니다. `country` 키에 다른 국가인 `미국` 을 전달합니다.


In [19]:
# chain2 를 실행합니다.
chain2.invoke({"country": "미국"})

'미국의 면적은 대략 9,833,520 제곱 킬로미터입니다.'

`combined` 객체의 `invoke` 메서드는 주어진 `country`에 대한 처리를 수행합니다.

이 예제에서는 `대한민국`라는 주제를 `invoke` 메서드에 전달하여 실행합니다.


In [20]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울이야.', 'area': '대한민국의 면적은 약 100,363제곱 킬로미터 입니다.'}

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다.
배치와 병렬 처리를 사용해 보도록 합시다.


`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.


In [21]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울입니다.', '미국의 수도는 워싱턴 D.C.입니다.']

`chain2.batch` 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 일괄 처리(batch)를 수행합니다.

이 예시에서는 `대한민국`와 `미국`라는 두 가지 국가에 대한 처리를 요청합니다.


In [22]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 면적은 약 100,363.90km² 입니다.', '미국의 면적은 약 9,834,000 제곱 킬로미터입니다.']

`combined.batch` 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 `대한민국`와 `미국` 두 나라에 대한 데이터를 배치 처리합니다.


In [23]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 면적은 약 100,363km² 입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 면적은 약 9,826,675 km² 입니다.'}]